## Introducción al diseño de estrategias de prompting



In [ ]:
# Librerias necesarias para el proyecto 
import os 
from dotenv import load_dotenv
from openai import OpenAI
from enum import Enum
from IPython.display import display, Markdown

In [ ]:
# Cargando las variables de entorno 

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# Diseñando una Clase para manejar diferentes tipos de modelos 
# Nos enfocaremos en OpenAI

class OpenAIModels(str,Enum): # Str -> Para que los valores sean cadenas de texto # Enum -> Para crear una enumeración
    GPT_4o = "gpt-4o"
    GPT_4o_mini = "gpt-4o-mini"
    GPT_5_mini = "gpt-5-mini"

MODEL = OpenAIModels.GPT_5_mini # Seleccionando el modelo a utilizar



In [ ]:
# Creamos una funcion wrapper para tener el llamado de OPenAI y sus componentes orquestados 

def get_completion(system_prompt, # Define el comportamiento del modelo
                   user_prompt,  # Es ;la solicitud del usuario
                   model=MODEL):

    # Construcción de los mensajes 
    messages = [
        {"role": "user", "content": user_prompt},
    ]
    if system_prompt is not None:
        messages = [
            {"role": "system", "content": system_prompt},
            *messages,
        ]
    try:
        # LLamado a la API de OpenAI
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=None, # Controla la creatividad de las respuestas
        )
        return response.choices[0].message.content # DEvuelve solo el cuerpo de la respuesta , no toda la metadata
    except Exception as e:
        return f"An error occurred: {e}"

In [ ]:
def display_responses(*args):
    """AYuda visual en formato markdown para comparar los modelos."""
    markdown_string = "<table><tr>"
    for arg in args:
        markdown_string += f"<th>System Prompt:<br />{arg['system_prompt']}<br /><br />"
        markdown_string += f"User Prompt:<br />{arg['user_prompt']}</th>"
    markdown_string += "</tr>"
    markdown_string += "<tr>"
    for arg in args:
        markdown_string += f"<td>Response:<br />{arg['response']}</td>"
    markdown_string += "</tr></table>"
    display(Markdown(markdown_string))

### Estrategias y comparativas de Prompting

In [ ]:
system_prompt = "Eres un asistente útil y creativo , en temas de coqueteo y relaciones amorosas respetuosas."
user_prompt = "Escribe un mensaje coqueto para invitar a salir a alguien que te gusta."

print('=='*32)
print(f'Enviar solicitud al modelo:  {MODEL}')
baseline_response = get_completion(system_prompt, user_prompt)
print('=='*32)
display_responses({
    "system_prompt": system_prompt,
    "user_prompt": user_prompt, 
    "response": baseline_response
})

### Agregandole un ROl más profesional

In [ ]:
role_system_prompt = "Eres un experto en seducción , relaciones amorosas y extremadamente latino. Usas la poesia y la mistica de la música para conquistar. Tambien eres un experto en cocina y sabes como combinar platos y bebidas para un fin romántico."

print("=="*32)
role_response = get_completion(role_system_prompt, user_prompt)
print("=="*32)
display_responses(
    {
        "system_prompt": system_prompt,
        "user_prompt": user_prompt,
        "response": baseline_response
    },  
    {
        "system_prompt": role_system_prompt,    
        "user_prompt": user_prompt,
        "response": role_response
    },
)

### Agregando restricciones

In [ ]:
constraints_system_prompt = f"""
                             {role_system_prompt}. Solo tienes tiempo para cocinar el Jueves a las 8 PM, porque después quitaran el gas en tu edificio. Como presupuesto para la cena tienes 50 dólares.
                             No recomiendes un postre , dado que por la hora no habrá tiempo para prepararlo.
                             """

print("=="*32)
constraints_response = get_completion(constraints_system_prompt, user_prompt)
print("=="*32)
display_responses(
    {
        "system_prompt": system_prompt,
        "user_prompt": user_prompt,
        "response": baseline_response
    },  
    {
        "system_prompt": role_system_prompt,    
        "user_prompt": user_prompt,
        "response": role_response
    },
    {
        "system_prompt": constraints_system_prompt,    
        "user_prompt": user_prompt, 
        "response": constraints_response
    },)